# View Hdf5 file

In [6]:
import h5py as h5

In [7]:
# view the tree structure of hdf5 file
def print_structure(name, obj):
    print(name)
    
## example: input_h5.visititems(print_structure)

In [24]:
case_name = "HFR_model_100x100x2_cyclic/"

model_dir = "/global/cscratch1/sd/pshuai/" + case_name
fname_pflotran_h5 = model_dir + "pflotran_100x100x2_cyclic_2014_2015_full.h5"
# fname_output_h5 = model_dir + "pflotran_100x100x2_6h_bc_120h_output.h5"

In [25]:
f = h5.File(fname_pflotran_h5, "r")

In [26]:
f.visititems(print_structure)

Coordinates
Coordinates/X [m]
Coordinates/Y [m]
Coordinates/Z [m]
Provenance
Provenance/PETSc
Provenance/PETSc/detail_petsc_config
Provenance/PETSc/detail_petsc_parent
Provenance/PETSc/detail_petsc_status
Provenance/PFLOTRAN
Provenance/PFLOTRAN/detail_pflotran_diff
Provenance/PFLOTRAN/detail_pflotran_fflags
Provenance/PFLOTRAN/detail_pflotran_parent
Provenance/PFLOTRAN/detail_pflotran_status
Provenance/PFLOTRAN/pflotran_compile_date_time
Provenance/PFLOTRAN/pflotran_compile_hostname
Provenance/PFLOTRAN/pflotran_compile_user
Provenance/PFLOTRAN/pflotran_input_file
Time:  6.17880E+04 h
Time:  6.17880E+04 h/Kludged_material_ids_for_VisIt
Time:  6.17880E+04 h/Liquid X-Flux Velocities
Time:  6.17880E+04 h/Liquid X-Velocity [m_per_h]
Time:  6.17880E+04 h/Liquid Y-Flux Velocities
Time:  6.17880E+04 h/Liquid Y-Velocity [m_per_h]
Time:  6.17880E+04 h/Liquid Z-Flux Velocities
Time:  6.17880E+04 h/Liquid Z-Velocity [m_per_h]
Time:  6.17880E+04 h/Liquid_Head [m]
Time:  6.17880E+04 h/Liquid_Pressur

In [11]:
f.close()